In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Reshape, Flatten
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = np.load('/content/drive/My Drive/test_colab/dataset_2_char_cat_45000_15000_140.npy')

In [4]:
data.shape

(60000, 14001)

In [4]:
# Train, Test Split
X, Y = data[:, 1:], data[:, 0]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.7, shuffle=True, random_state=11)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(42000, 14000) (42000,)
(18000, 14000) (18000,)


In [0]:
Y_train_cnn = to_categorical(Y_train)
Y_test_cnn = to_categorical(Y_test)

In [6]:
cae_input = Input((X_train.shape[1], ))  # adapt this if using `channels_first` image data format
H = Reshape((100, 140, 1))(cae_input)

# Encoded
H = Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu')(H)
H = MaxPooling2D((2,2))(H)
H = Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu')(H)
H = MaxPooling2D((2,2))(H)

# Decoded

H = Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu')(H)
H = UpSampling2D((2, 2))(H)
H = Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu')(H)
H = UpSampling2D((2, 2))(H)
H = Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu')(H)
H = Conv2D(filters=1, kernel_size=(1, 1), padding='same', activation='relu')(H)

cae_output = Flatten()(H)
cae_model = Model(cae_input,cae_output)
cae_model.summary()








Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 14000)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 100, 140, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 140, 32)      160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 70, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 70, 32)        4128      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 35, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 35, 32)       

In [13]:
%%time
adam = Adam(lr = 1e-04)
cae_model.compile(loss='categorical_crossentropy', 
                  optimizer= adam, metrics=['accuracy'])

CPU times: user 35.6 ms, sys: 0 ns, total: 35.6 ms
Wall time: 35.7 ms


In [14]:
for i, l in enumerate(cae_model.layers):
    l.trainable = False
    l.name = 'cae_'+str(i)
cae_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cae_0 (InputLayer)           (None, 14000)             0         
_________________________________________________________________
cae_1 (Reshape)              (None, 100, 140, 1)       0         
_________________________________________________________________
cae_2 (Conv2D)               (None, 100, 140, 32)      160       
_________________________________________________________________
cae_3 (MaxPooling2D)         (None, 50, 70, 32)        0         
_________________________________________________________________
cae_4 (Conv2D)               (None, 50, 70, 32)        4128      
_________________________________________________________________
cae_5 (MaxPooling2D)         (None, 25, 35, 32)        0         
_________________________________________________________________
cae_6 (Conv2D)               (None, 25, 35, 32)        4128

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
